idea -> 

murmur 

학습 라벨
murmur 0 1 
unknown 0.5 0.5
absent 1 0
murmur 다른거 0.5 0.5

합칠 때: 
murmur 가 가장 높은 확률인 벡터 뽑기
murmur 확률 과 0, 0.5, 1 과의 거리의 역수를 합이 1이 되게 표준화시켜 점수만들기

학습라벨
- normal 1 0
- abnormal 
present 0.1 0.9
unknown 0.15 0.85
absent 0.2 0.8
합칠 때: 평균



In [1]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display


In [2]:
import sys
sys.path.insert(0,'/home/ikwak2/hmd/notebooks')
#sys.path.insert(0,'/home/ikwak2/hmd/iy_classifier')
sys.path.insert(0,'/home/ikwak2/hmd/notebooks/utils')


In [3]:
from helper_code import *
from get_feature import *

In [5]:
root_dir = '/home/ikwak2/hmd/physionet.org/files/circor-heart-sound/1.0.3'
training_data_file = root_dir + '/' + 'training_data.csv'
training_data_dir = root_dir + '/' + 'training_data'
model_dir = root_dir + '/' + 'model'

In [6]:
import tensorflow as tf

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


Fri Jul  1 23:12:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   33C    P8    13W /

In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

In [8]:
df = pd.read_csv(training_data_file)
df[:10]

,Patient ID,Recording locations:,Age,Sex,Height,Weight,Pregnancy status,Murmur,Murmur locations,Most audible location,...,Systolic murmur pitch,Systolic murmur quality,Diastolic murmur timing,Diastolic murmur shape,Diastolic murmur grading,Diastolic murmur pitch,Diastolic murmur quality,Outcome,Campaign,Additional ID
0,2530,AV+PV+TV+MV,Child,Female,98.0,15.90,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
1,9979,AV+PV+TV+MV,Child,Female,103.0,13.10,False,Present,AV+MV+PV+TV,TV,...,High,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
2,9983,AV+PV+TV+MV,Child,Male,115.0,19.10,False,Unknown,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
3,13918,AV+PV+TV+MV,Child,Male,98.0,15.90,False,Present,TV,TV,...,Low,Blowing,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
4,14241,AV+PV+TV+MV,Child,Male,87.0,11.20,False,Present,AV+MV+PV+TV,PV,...,Low,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
5,14998,AV+PV+TV+MV,Child,Male,NaN,NaN,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
6,23625,AV+PV+TV+MV,Child,Female,92.0,14.00,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,50379.0
7,24160,AV+PV+TV+MV,Child,Female,98.0,17.66,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
8,29045,AV+PV+TV+MV,Child,Female,88.0,12.50,False,Present,AV+MV+PV+TV,TV,...,Low,Blowing,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
9,29378,AV+MV,Child,Female,82.0,10.70,False,Unknown,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN


In [9]:
data_folder =  '/home/ikwak2/hmd/physionet.org/files/circor-heart-sound/1.0.3/training_data'
train_folder =  '/home/ubuntu/data/hmd/murmur/train'
test_folder = '/home/ubuntu/data/hmd/murmur/test'
model_folder = 'tmp_model2'

In [10]:
data_folder

'/home/ikwak2/hmd/physionet.org/files/circor-heart-sound/1.0.3/training_data'

In [11]:
patient_files = find_patient_files(data_folder)
patient_files_trn = find_patient_files(train_folder)
patient_files_test = find_patient_files(test_folder)

num_patient_files = len(patient_files)


In [12]:
!cat /home/ubuntu/data/hmd/murmur/train/46778.txt

46778 1 4000
MV 46778_MV.hea 46778_MV.wav 46778_MV.tsv
#Age: Adolescent
#Sex: Female
#Height: 150.0
#Weight: 54.7
#Pregnancy status: False
#Murmur: Present
#Murmur locations: MV
#Most audible location: MV
#Systolic murmur timing: Holosystolic
#Systolic murmur shape: Plateau
#Systolic murmur grading: I/VI
#Systolic murmur pitch: Low
#Systolic murmur quality: Harsh
#Diastolic murmur timing: Early-diastolic
#Diastolic murmur shape: Decrescendo
#Diastolic murmur grading: I/IV
#Diastolic murmur pitch: Low
#Diastolic murmur quality: Blowing
#Outcome: Abnormal
#Campaign: CC2015
#Additional ID: 49754


In [13]:
!cat /home/ubuntu/data/hmd/murmur/train/46532.txt

46532 1 4000
AV 46532_AV.hea 46532_AV.wav 46532_AV.tsv
#Age: Child
#Sex: Female
#Height: 70.0
#Weight: 9.0
#Pregnancy status: False
#Murmur: Absent
#Murmur locations: nan
#Most audible location: nan
#Systolic murmur timing: nan
#Systolic murmur shape: nan
#Systolic murmur grading: nan
#Systolic murmur pitch: nan
#Systolic murmur quality: nan
#Diastolic murmur timing: nan
#Diastolic murmur shape: nan
#Diastolic murmur grading: nan
#Diastolic murmur pitch: nan
#Diastolic murmur quality: nan
#Outcome: Normal
#Campaign: CC2015
#Additional ID: nan


In [14]:
len(patient_files)

942

In [15]:
os.makedirs(model_folder, exist_ok=True)

murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Normal', 'Abnormal']
num_mm_classes = len(murmur_classes)
num_o_classes = len(outcome_classes)

In [16]:
    features = dict()
    features['id'] = []
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
    features['mel1'] = []
    labels = []

    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)


In [44]:
#    for i in range(num_patient_files):

i = 1

current_patient_data = load_patient_data(patient_files_trn[i])
num_locations = get_num_locations(current_patient_data)
recording_information = current_patient_data.split('\n')[1:num_locations+1]
num_locations

4

In [27]:
recording_information

['AV 9979_AV.hea 9979_AV.wav 9979_AV.tsv',
 'PV 9979_PV.hea 9979_PV.wav 9979_PV.tsv',
 'TV 9979_TV.hea 9979_TV.wav 9979_TV.tsv',
 'MV 9979_MV.hea 9979_MV.wav 9979_MV.tsv']

In [28]:
current_patient_data

'9979 4 4000\nAV 9979_AV.hea 9979_AV.wav 9979_AV.tsv\nPV 9979_PV.hea 9979_PV.wav 9979_PV.tsv\nTV 9979_TV.hea 9979_TV.wav 9979_TV.tsv\nMV 9979_MV.hea 9979_MV.wav 9979_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 103.0\n#Weight: 13.1\n#Pregnancy status: False\n#Murmur: Present\n#Murmur locations: AV+MV+PV+TV\n#Most audible location: TV\n#Systolic murmur timing: Holosystolic\n#Systolic murmur shape: Diamond\n#Systolic murmur grading: III/VI\n#Systolic murmur pitch: High\n#Systolic murmur quality: Harsh\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Abnormal\n#Campaign: CC2015\n#Additional ID: nan\n'

In [29]:
#        for j in range(num_locations) :
j = 0
entries = recording_information[j].split(' ')
recording_file = entries[2]
filename = os.path.join(data_folder, recording_file)

# Extract id
id1 = recording_file.split('_')[0]
features['id'].append(id1)

# Extract melspec
mel1 = feature_extract_melspec(filename)[0]
features['mel1'].append(mel1)

# Extract age_group
age_group = get_age(current_patient_data)
current_age_group = np.zeros(6, dtype=int)
if age_group in age_classes:
    j = age_classes.index(age_group)
    current_age_group[j] = 1
else :
    current_age_group[5] = 1
features['age'].append(current_age_group)

# Extract sex
sex = get_sex(current_patient_data)
sex_features = np.zeros(2, dtype=int)
if compare_strings(sex, 'Female'):
    sex_features[0] = 1
elif compare_strings(sex, 'Male'):
    sex_features[1] = 1
features['sex'].append(sex_features)

# Extract height and weight.
height = get_height(current_patient_data)
weight = get_weight(current_patient_data)
features['hw'].append(np.array([height, weight]))

# Extract pregnancy
is_pregnant = get_pregnancy_status(current_patient_data)
features['preg'].append(is_pregnant)

# Extract location
locations = entries[0]
num_recording_locations = len(recording_locations)
loc_features = np.zeros(num_recording_locations)
if locations in recording_locations:
    j = recording_locations.index(locations)
    loc_features[j] = 1
features['loc'].append(loc_features)


In [30]:
locations

'AV'

In [39]:
def get_murmur_loc(data):
    murmur_loc = 0
    for l in data.split('\n'):
        if l.startswith('#Murmur locations: '):
            try:
                murmur_loc = l.split(': ')[1]
#                murmur_loc = murmur_loc.split('+')
            except:
                pass
    return murmur_loc

In [45]:
dd = get_murmur_loc(current_patient_data)

In [46]:
dd

'AV+MV+PV+TV'

In [34]:
current_patient_data.split('\n')

['9979 4 4000',
 'AV 9979_AV.hea 9979_AV.wav 9979_AV.tsv',
 'PV 9979_PV.hea 9979_PV.wav 9979_PV.tsv',
 'TV 9979_TV.hea 9979_TV.wav 9979_TV.tsv',
 'MV 9979_MV.hea 9979_MV.wav 9979_MV.tsv',
 '#Age: Child',
 '#Sex: Female',
 '#Height: 103.0',
 '#Weight: 13.1',
 '#Pregnancy status: False',
 '#Murmur: Present',
 '#Murmur locations: AV+MV+PV+TV',
 '#Most audible location: TV',
 '#Systolic murmur timing: Holosystolic',
 '#Systolic murmur shape: Diamond',
 '#Systolic murmur grading: III/VI',
 '#Systolic murmur pitch: High',
 '#Systolic murmur quality: Harsh',
 '#Diastolic murmur timing: nan',
 '#Diastolic murmur shape: nan',
 '#Diastolic murmur grading: nan',
 '#Diastolic murmur pitch: nan',
 '#Diastolic murmur quality: nan',
 '#Outcome: Abnormal',
 '#Campaign: CC2015',
 '#Additional ID: nan',
 '']

In [47]:
recording_locations

['AV', 'MV', 'PV', 'TV', 'PhC']

In [ ]:
mm_label = get_murmur(current_patient_data)
out_label = get_outcome(current_patient_data)

In [57]:
current_mm_labels = np.zeros(2)
current_out_labels = np.zeros(2)
if mm_label == 'Absent' :
    current_mm_labels = np.array([1, 0])
elif mm_label == 'unknown' :
    current_mm_labels = np.array([1, 0])
else :
    mm_loc = get_murmur_loc(current_patient_data)
    if mm_loc == 'nan' :
        current_mm_labels = np.array([0.2, 0.8])
    else :
        mm_loc = mm_loc.split('+')
        if location in mm_loc :
            current_mm_labels = np.array([0, 1])
        else :
            current_mm_labels = np.array([0.4, 0.6])

if out_label == 'Abnormal' :
    current_out_labels = np.array([1, 0])
else :
    current_out_labels = np.array([0, 1])
            

In [55]:
out_label

'Abnormal'

In [25]:
mm_label = get_murmur(current_patient_data)
out_label = get_outcome(current_patient_data)
mm_label

'Absent'

In [48]:
j = recording_locations.index(locations)

In [50]:
locations

'AV'

In [51]:
mm_loc = get_murmur_loc(current_patient_data)

In [53]:
mm_loc = mm_loc.split('+')

In [54]:
mm_loc

['AV', 'MV', 'PV', 'TV']

In [ ]:

current_labels = np.zeros(2)
label = get_label(current_patient_data)
if label in classes:
    j = classes.index(label)
    current_labels[j] = 1
labels.append(current_labels)


In [58]:
    mm_labels = np.zeros((num_patient_files,2))
    out_labels = np.zeros((num_patient_files,2))


In [41]:
    features = dict()
    features['id'] = []
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
    features['mel1'] = []
    mm_labels = []
    out_labels = []

    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)
#    mm_labels = np.zeros((num_patient_files,2))
#    out_labels = np.zeros((num_patient_files,2))

    for i in range(num_patient_files):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)

            # Extract id
            id1 = recording_file.split('_')[0]
            features['id'].append(id1)

            # Extract melspec
            mel1 = feature_extract_melspec(filename)[0]
            features['mel1'].append(mel1)

            # Extract age_group
            age_group = get_age(current_patient_data)
            current_age_group = np.zeros(6, dtype=int)
            if age_group in age_classes:
                j = age_classes.index(age_group)
                current_age_group[j] = 1
            else :
                current_age_group[5] = 1
            features['age'].append(current_age_group)

            # Extract sex
            sex = get_sex(current_patient_data)
            sex_features = np.zeros(2, dtype=int)
            if compare_strings(sex, 'Female'):
                sex_features[0] = 1
            elif compare_strings(sex, 'Male'):
                sex_features[1] = 1
            features['sex'].append(sex_features)

            # Extract height and weight.
            height = get_height(current_patient_data)
            weight = get_weight(current_patient_data)
            features['hw'].append(np.array([height, weight]))

            # Extract pregnancy
            is_pregnant = get_pregnancy_status(current_patient_data)
            features['preg'].append(is_pregnant)

            # Extract location
            locations = entries[0]
            num_recording_locations = len(recording_locations)
            loc_features = np.zeros(num_recording_locations)
            if locations in recording_locations:
                j = recording_locations.index(locations)
                loc_features[j] = 1
            features['loc'].append(loc_features)

            # Extract labels 
#            current_labels = np.zeros(2)
#            label = get_label(current_patient_data)
#            if label in classes:
#                j = classes.index(label)
#                current_labels[j] = 1
#            labels.append(current_labels)
            mm_label = get_murmur(current_patient_data)
            out_label = get_outcome(current_patient_data)
            current_mm_labels = np.zeros(2)
            current_out_labels = np.zeros(2)
            if mm_label == 'Absent' :
                current_mm_labels = np.array([1, 0])
            elif mm_label == 'unknown' :
                current_mm_labels = np.array([1, 0])
            else :
                mm_loc = get_murmur_loc(current_patient_data)
                if mm_loc == 'nan' :
                    current_mm_labels = np.array([0.2, 0.8])
                else :
                    mm_loc = mm_loc.split('+')
                    if location in mm_loc :
                        current_mm_labels = np.array([0, 1])
                    else :
                        current_mm_labels = np.array([0.4, 0.6])

            if out_label == 'Abnormal' :
                current_out_labels = np.array([1, 0])
            else :
                current_out_labels = np.array([0, 1])
                
            mm_labels.append(current_mm_labels)
            out_labels.append(current_out_labels)

    M, N = features['mel1'][i].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)
    
    for k1 in features.keys() :
        features[k1] = np.array(features[k1])
    
    return features, mm_labels, out_labels

In [153]:
entries

['AV', '9979_AV.hea', '9979_AV.wav', '9979_AV.tsv']

In [154]:
def get_features(patient_files_trn) :
    features = dict()
    features['id'] = []
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
    features['mel1'] = []
#    labels = []
    mm_labels = []
    out_labels = []

    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)

    for i in range(num_patient_files):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)

            # Extract id
            id1 = recording_file.split('_')[0]
            features['id'].append(id1)

            # Extract melspec
            mel1 = feature_extract_melspec(filename)[0]
            features['mel1'].append(mel1)

            # Extract age_group
            age_group = get_age(current_patient_data)
            current_age_group = np.zeros(6, dtype=int)
            if age_group in age_classes:
                j = age_classes.index(age_group)
                current_age_group[j] = 1
            else :
                current_age_group[5] = 1
            features['age'].append(current_age_group)

            # Extract sex
            sex = get_sex(current_patient_data)
            sex_features = np.zeros(2, dtype=int)
            if compare_strings(sex, 'Female'):
                sex_features[0] = 1
            elif compare_strings(sex, 'Male'):
                sex_features[1] = 1
            features['sex'].append(sex_features)

            # Extract height and weight.
            height = get_height(current_patient_data)
            weight = get_weight(current_patient_data)
            features['hw'].append(np.array([height, weight]))

            # Extract pregnancy
            is_pregnant = get_pregnancy_status(current_patient_data)
            features['preg'].append(is_pregnant)

            # Extract location
            locations = entries[0]
            num_recording_locations = len(recording_locations)
            loc_features = np.zeros(num_recording_locations)
            if locations in recording_locations:
                j = recording_locations.index(locations)
                loc_features[j] = 1
            features['loc'].append(loc_features)

            # Extract labels 
            mm_label = get_murmur(current_patient_data)
            out_label = get_outcome(current_patient_data)
            current_mm_labels = np.zeros(2)
            current_out_labels = np.zeros(2)
            if mm_label == 'Absent' :
                current_mm_labels = np.array([1, 0])
            elif mm_label == 'unknown' :
                current_mm_labels = np.array([1, 0])
            else :
                mm_loc = get_murmur_loc(current_patient_data)
                if mm_loc == 'nan' :
                    current_mm_labels = np.array([0.2, 0.8])
                else :
                    mm_loc = mm_loc.split('+')
                    if locations in mm_loc :
                        current_mm_labels = np.array([0, 1])
                    else :
                        current_mm_labels = np.array([0.4, 0.6])

            if out_label == 'Normal' :
                current_out_labels = np.array([1, 0])
            else :
                current_out_labels = np.array([0, 1])
                
            mm_labels.append(current_mm_labels)
            out_labels.append(current_out_labels)

    M, N = features['mel1'][i].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)
    
    for k1 in features.keys() :
        features[k1] = np.array(features[k1])
    
    mm_labels = np.array(mm_labels)
    out_labels = np.array(out_labels)
    return features, mm_labels, out_labels

In [218]:
def get_features_3lb(patient_files_trn) :
    features = dict()
    features['id'] = []
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
    features['mel1'] = []
#    labels = []
    mm_labels = []
    out_labels = []

    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)

    for i in range(num_patient_files):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)

            # Extract id
            id1 = recording_file.split('_')[0]
            features['id'].append(id1)

            # Extract melspec
            mel1 = feature_extract_melspec(filename)[0]
            features['mel1'].append(mel1)

            # Extract age_group
            age_group = get_age(current_patient_data)
            current_age_group = np.zeros(6, dtype=int)
            if age_group in age_classes:
                j = age_classes.index(age_group)
                current_age_group[j] = 1
            else :
                current_age_group[5] = 1
            features['age'].append(current_age_group)

            # Extract sex
            sex = get_sex(current_patient_data)
            sex_features = np.zeros(2, dtype=int)
            if compare_strings(sex, 'Female'):
                sex_features[0] = 1
            elif compare_strings(sex, 'Male'):
                sex_features[1] = 1
            features['sex'].append(sex_features)

            # Extract height and weight.
            height = get_height(current_patient_data)
            weight = get_weight(current_patient_data)
            features['hw'].append(np.array([height, weight]))

            # Extract pregnancy
            is_pregnant = get_pregnancy_status(current_patient_data)
            features['preg'].append(is_pregnant)

            # Extract location
            locations = entries[0]
            num_recording_locations = len(recording_locations)
            loc_features = np.zeros(num_recording_locations)
            if locations in recording_locations:
                j = recording_locations.index(locations)
                loc_features[j] = 1
            features['loc'].append(loc_features)

            # Extract labels 
            mm_label = get_murmur(current_patient_data)
            out_label = get_outcome(current_patient_data)
            current_mm_labels = np.zeros(2)
            current_out_labels = np.zeros(2)
            if mm_label == 'Absent' :
                current_mm_labels = np.array([1, 0, 0])
            elif mm_label == 'unknown' :
                current_mm_labels = np.array([0, 1, 0])
            else :
                mm_loc = get_murmur_loc(current_patient_data)
                if mm_loc == 'nan' :
                    current_mm_labels = np.array([0.05, 0.15, 0.8])
                else :
                    mm_loc = mm_loc.split('+')
                    if locations in mm_loc :
                        current_mm_labels = np.array([0, 0, 1])
                    else :
                        current_mm_labels = np.array([0.2, 0.3, 0.5])

            if out_label == 'Normal' :
                current_out_labels = np.array([1, 0])
            else :
                if mm_label == 'Absent' :
                    current_out_labels = np.array([0.2, 0.8])
                elif mm_label == 'unknown' :
                    current_out_labels = np.array([0.15, 0.85])
                else :
                    current_out_labels = np.array([0.05, 0.95])
                
            mm_labels.append(current_mm_labels)
            out_labels.append(current_out_labels)

    M, N = features['mel1'][i].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)
    
    for k1 in features.keys() :
        features[k1] = np.array(features[k1])
    
    mm_labels = np.array(mm_labels)
    out_labels = np.array(out_labels)
    return features, mm_labels, out_labels

In [61]:

def get_features_wo_labels(patient_files_trn) :
    features = dict()
    features['id'] = []
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
    features['mel1'] = []
    labels = []

    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)

    for i in range(num_patient_files):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)

            # Extract id
            id1 = recording_file.split('_')[0]
            features['id'].append(id1)

            # Extract melspec
            mel1 = feature_extract_melspec(filename)[0]
            features['mel1'].append(mel1)

            # Extract age_group
            age_group = get_age(current_patient_data)
            current_age_group = np.zeros(6, dtype=int)
            if age_group in age_classes:
                j = age_classes.index(age_group)
                current_age_group[j] = 1
            else :
                current_age_group[5] = 1
            features['age'].append(current_age_group)

            # Extract sex
            sex = get_sex(current_patient_data)
            sex_features = np.zeros(2, dtype=int)
            if compare_strings(sex, 'Female'):
                sex_features[0] = 1
            elif compare_strings(sex, 'Male'):
                sex_features[1] = 1
            features['sex'].append(sex_features)

            # Extract height and weight.
            height = get_height(current_patient_data)
            weight = get_weight(current_patient_data)
            features['hw'].append(np.array([height, weight]))

            # Extract pregnancy
            is_pregnant = get_pregnancy_status(current_patient_data)
            features['preg'].append(is_pregnant)

            # Extract location
            locations = entries[0]
            num_recording_locations = len(recording_locations)
            loc_features = np.zeros(num_recording_locations)
            if locations in recording_locations:
                j = recording_locations.index(locations)
                loc_features[j] = 1
            features['loc'].append(loc_features)
        
    M, N = features['mel1'][i].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)

    for k1 in features.keys() :
        features[k1] = np.array(features[k1])
    
    return features

In [219]:
features_trn = get_features_3lb(patient_files_trn)

In [220]:
features_test = get_features_3lb(patient_files_test)

In [26]:
shuffled_idx = np.random.permutation(942)

In [14]:
patient_files_trn = patient_files[:800]
patient_files_test = patient_files[800:]
num_patient_files = len(patient_files_trn)

In [15]:
os.makedirs(model_folder, exist_ok=True)

classes = ['Present', 'Unknown', 'Absent']
num_classes = len(classes)


In [16]:
def get_features(patient_files_trn) :
    features = dict()
    features['id'] = []
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
    features['mel1'] = []
    labels = []

    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)

    for i in range(num_patient_files):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)

            # Extract id
            id1 = recording_file.split('_')[0]
            features['id'].append(id1)

            # Extract melspec
            mel1 = feature_extract_melspec(filename)[0]
            features['mel1'].append(mel1)

            # Extract age_group
            age_group = get_age(current_patient_data)
            current_age_group = np.zeros(6, dtype=int)
            if age_group in age_classes:
                j = age_classes.index(age_group)
                current_age_group[j] = 1
            else :
                current_age_group[5] = 1
            features['age'].append(current_age_group)

            # Extract sex
            sex = get_sex(current_patient_data)
            sex_features = np.zeros(2, dtype=int)
            if compare_strings(sex, 'Female'):
                sex_features[0] = 1
            elif compare_strings(sex, 'Male'):
                sex_features[1] = 1
            features['sex'].append(sex_features)

            # Extract height and weight.
            height = get_height(current_patient_data)
            weight = get_weight(current_patient_data)
            features['hw'].append(np.array([height, weight]))

            # Extract pregnancy
            is_pregnant = get_pregnancy_status(current_patient_data)
            features['preg'].append(is_pregnant)

            # Extract location
            locations = entries[0]
            num_recording_locations = len(recording_locations)
            loc_features = np.zeros(num_recording_locations)
            if locations in recording_locations:
                j = recording_locations.index(locations)
                loc_features[j] = 1
            features['loc'].append(loc_features)

            # Extract labels and use one-hot encoding.
            current_labels = np.zeros(num_classes, dtype=int)
            label = get_label(current_patient_data)
            if label in classes:
                j = classes.index(label)
                current_labels[j] = 1
            labels.append(current_labels)
        
    M, N = features['mel1'][i].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)

    labels = np.array(labels)
    
    for k1 in features.keys() :
        features[k1] = np.array(features[k1])
    
    return features, labels

In [17]:

def get_features_wo_labels(patient_files_trn) :
    features = dict()
    features['id'] = []
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
    features['mel1'] = []
    labels = []

    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)

    for i in range(num_patient_files):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)

            # Extract id
            id1 = recording_file.split('_')[0]
            features['id'].append(id1)

            # Extract melspec
            mel1 = feature_extract_melspec(filename)[0]
            features['mel1'].append(mel1)

            # Extract age_group
            age_group = get_age(current_patient_data)
            current_age_group = np.zeros(6, dtype=int)
            if age_group in age_classes:
                j = age_classes.index(age_group)
                current_age_group[j] = 1
            else :
                current_age_group[5] = 1
            features['age'].append(current_age_group)

            # Extract sex
            sex = get_sex(current_patient_data)
            sex_features = np.zeros(2, dtype=int)
            if compare_strings(sex, 'Female'):
                sex_features[0] = 1
            elif compare_strings(sex, 'Male'):
                sex_features[1] = 1
            features['sex'].append(sex_features)

            # Extract height and weight.
            height = get_height(current_patient_data)
            weight = get_weight(current_patient_data)
            features['hw'].append(np.array([height, weight]))

            # Extract pregnancy
            is_pregnant = get_pregnancy_status(current_patient_data)
            features['preg'].append(is_pregnant)

            # Extract location
            locations = entries[0]
            num_recording_locations = len(recording_locations)
            loc_features = np.zeros(num_recording_locations)
            if locations in recording_locations:
                j = recording_locations.index(locations)
                loc_features[j] = 1
            features['loc'].append(loc_features)
        
    M, N = features['mel1'][i].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)

    for k1 in features.keys() :
        features[k1] = np.array(features[k1])
    
    return features

In [18]:
features_trn = get_features(patient_files_trn)

In [83]:
len(features_trn[0]['age'])

2532

In [19]:
features_test = get_features(patient_files_test)

In [84]:
len(features_test[0]['age'])

631

In [146]:
features_trn[2][:50]

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0]])

In [25]:
i = 0
current_patient_data = load_patient_data(patient_files_trn[i])

In [26]:
current_patient_data

'2530 4 4000\nAV 2530_AV.hea 2530_AV.wav 2530_AV.tsv\nPV 2530_PV.hea 2530_PV.wav 2530_PV.tsv\nTV 2530_TV.hea 2530_TV.wav 2530_TV.tsv\nMV 2530_MV.hea 2530_MV.wav 2530_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 98.0\n#Weight: 15.9\n#Pregnancy status: False\n#Murmur: Absent\n#Murmur locations: nan\n#Most audible location: nan\n#Systolic murmur timing: nan\n#Systolic murmur shape: nan\n#Systolic murmur grading: nan\n#Systolic murmur pitch: nan\n#Systolic murmur quality: nan\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Campaign: CC2015\n#Additional ID: nan'

In [ ]:
# Load recordings. wav data recording
def load_recordings(data_folder, data, get_frequencies=False):
    num_locations = get_num_locations(data)
    recording_information = data.split('\n')[1:num_locations+1]

    recordings = list()
    frequencies = list()
    for i in range(num_locations):
        entries = recording_information[i].split(' ')
        recording_file = entries[2]
        filename = os.path.join(data_folder, recording_file)
        recording, frequency = load_wav_file(filename)
        recordings.append(recording)
        frequencies.append(frequency)

    if get_frequencies:
        return recordings, frequencies
    else:
        return recordings

In [27]:
num_locations = get_num_locations(current_patient_data)
recording_information = current_patient_data.split('\n')[1:num_locations+1]

In [28]:
print(num_locations, recording_information)


4 ['AV 2530_AV.hea 2530_AV.wav 2530_AV.tsv', 'PV 2530_PV.hea 2530_PV.wav 2530_PV.tsv', 'TV 2530_TV.hea 2530_TV.wav 2530_TV.tsv', 'MV 2530_MV.hea 2530_MV.wav 2530_MV.tsv']


In [29]:
recording_information

['AV 2530_AV.hea 2530_AV.wav 2530_AV.tsv',
 'PV 2530_PV.hea 2530_PV.wav 2530_PV.tsv',
 'TV 2530_TV.hea 2530_TV.wav 2530_TV.tsv',
 'MV 2530_MV.hea 2530_MV.wav 2530_MV.tsv']

In [30]:
entries = recording_information[i].split(' ')
entries

['AV', '2530_AV.hea', '2530_AV.wav', '2530_AV.tsv']

In [31]:
recording_file = entries[2]

In [32]:
recording_file

'2530_AV.wav'

In [33]:
filename = os.path.join(data_folder, recording_file)
filename

'/home/ubuntu/hmd/notebooks/physionet.org/files/circor-heart-sound/1.0.1/training_data/2530_AV.wav'

In [35]:
features = dict()

In [36]:
features = dict()
features['id'] = []
features['age'] = []
features['sex'] = []
features['hw'] = []
features['preg'] = []
features['loc'] = []
features['mel1'] = []


In [37]:
id1 = recording_file.split('_')[0]
features['id'].append(id1)

In [38]:
mel1 = feature_extract_melspec(filename)[0]
mel1.shape

(100, 313)

In [39]:
features['mel1'].append(mel1)

In [40]:
features

{'id': ['2530'],
 'age': [],
 'sex': [],
 'hw': [],
 'preg': [],
 'loc': [],
 'mel1': [array([[-14.24342  , -14.460995 , -12.48743  , ...,  -8.350793 ,
           -9.550913 ,  -6.6099434],
         [ -8.833427 , -11.78526  , -10.376642 , ..., -10.638776 ,
          -14.766047 ,  -2.595482 ],
         [ -7.201795 , -11.942595 , -14.396763 , ..., -13.780443 ,
          -11.831632 , -10.3140745],
         ...,
         [-43.652073 , -60.107727 , -58.363945 , ..., -46.318718 ,
          -47.93747  , -47.206367 ],
         [-42.097    , -59.548874 , -57.106033 , ..., -45.140682 ,
          -53.84076  , -53.963337 ],
         [-42.240356 , -60.107727 , -60.107727 , ..., -51.811592 ,
          -55.905552 , -57.170105 ]], dtype=float32)]}

In [41]:
import tensorflow as tf

In [42]:
from tensorflow.keras.utils import to_categorical

In [44]:
entries_age = ['Child']
age_group = get_age(current_patient_data)

In [45]:
age_group

'Child'

In [47]:
age_group = get_age(current_patient_data)

age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
current_age_group = np.zeros(6, dtype=int)

if age_group in age_classes:
    j = age_classes.index(age_group)
    current_age_group[j] = 1
else :
    current_age_group[5] = 1



In [48]:
age_group

'Child'

In [49]:
current_age_group

array([0, 0, 1, 0, 0, 0])

In [51]:
age_group = 'ddd'

age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
current_age_group = np.zeros(6, dtype=int)

if age_group in age_classes:
    j = age_classes.index(age_group)
    current_age_group[j] = 1
else :
    current_age_group[5] = 1

In [52]:
current_age_group

array([0, 0, 0, 0, 0, 1])

In [53]:
features['age'].append(current_age_group)

In [54]:
# Extract sex. Use one-hot encoding.
sex = get_sex(current_patient_data)
sex_features = np.zeros(2, dtype=int)
if compare_strings(sex, 'Female'):
    sex_features[0] = 1
elif compare_strings(sex, 'Male'):
    sex_features[1] = 1

In [55]:
features['sex'].append(sex_features)

In [56]:
# Extract height and weight.
height = get_height(current_patient_data)
weight = get_weight(current_patient_data)
features['hw'].append(np.array([height, weight]))

In [57]:
is_pregnant = get_pregnancy_status(current_patient_data)
features['preg'].append(is_pregnant)

In [58]:
locations = entries[0]

recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
num_recording_locations = len(recording_locations)
loc_features = np.zeros(num_recording_locations)
if locations in recording_locations:
    j = recording_locations.index(locations)
    loc_features[j] = 1

In [59]:
locations

'AV'

In [60]:
loc_features

array([1., 0., 0., 0., 0.])

In [61]:
features['loc'].append(loc_features)

In [62]:
features

{'id': ['2530'],
 'age': [array([0, 0, 0, 0, 0, 1])],
 'sex': [array([1, 0])],
 'hw': [array([98. , 15.9])],
 'preg': [False],
 'loc': [array([1., 0., 0., 0., 0.])],
 'mel1': [array([[-14.24342  , -14.460995 , -12.48743  , ...,  -8.350793 ,
           -9.550913 ,  -6.6099434],
         [ -8.833427 , -11.78526  , -10.376642 , ..., -10.638776 ,
          -14.766047 ,  -2.595482 ],
         [ -7.201795 , -11.942595 , -14.396763 , ..., -13.780443 ,
          -11.831632 , -10.3140745],
         ...,
         [-43.652073 , -60.107727 , -58.363945 , ..., -46.318718 ,
          -47.93747  , -47.206367 ],
         [-42.097    , -59.548874 , -57.106033 , ..., -45.140682 ,
          -53.84076  , -53.963337 ],
         [-42.240356 , -60.107727 , -60.107727 , ..., -51.811592 ,
          -55.905552 , -57.170105 ]], dtype=float32)]}

In [ ]:

recording_information = current_patient_data.split('\n')[1:num_locations+1]

In [132]:

    current_patient_data = load_patient_data(patient_files[i])
    current_recordings = load_recordings(data_folder, current_patient_data)

In [135]:
current_patient_data

'2530 4 4000\nAV 2530_AV.hea 2530_AV.wav 2530_AV.tsv\nPV 2530_PV.hea 2530_PV.wav 2530_PV.tsv\nTV 2530_TV.hea 2530_TV.wav 2530_TV.tsv\nMV 2530_MV.hea 2530_MV.wav 2530_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 98.0\n#Weight: 15.9\n#Pregnancy status: False\n#Murmur: Absent\n#Murmur locations: nan\n#Most audible location: nan\n#Systolic murmur timing: nan\n#Systolic murmur shape: nan\n#Systolic murmur grading: nan\n#Systolic murmur pitch: nan\n#Systolic murmur quality: nan\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Campaign: CC2015\n#Additional ID: nan'

In [136]:
num_patient_files

800

In [143]:
features = dict()
features['id'] = []
features['age'] = []
features['sex'] = []
features['hw'] = []
features['preg'] = []
features['loc'] = []
features['mel1'] = []
labels = []

age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

for i in range(num_patient_files):

    # Load the current patient data and recordings.
    current_patient_data = load_patient_data(patient_files_trn[i])
    num_locations = get_num_locations(current_patient_data)
    recording_information = current_patient_data.split('\n')[1:num_locations+1]
    for j in range(num_locations) :
        entries = recording_information[j].split(' ')
        recording_file = entries[2]
        filename = os.path.join(data_folder, recording_file)
        
        # Extract id
        id1 = recording_file.split('_')[0]
        features['id'].append(id1)
        
        # Extract melspec
        mel1 = feature_extract_melspec(filename)[0]
        features['mel1'].append(mel1)
        
        # Extract age_group
        age_group = get_age(current_patient_data)
        current_age_group = np.zeros(6, dtype=int)
        if age_group in age_classes:
            j = age_classes.index(age_group)
            current_age_group[j] = 1
        else :
            current_age_group[5] = 1
        features['age'].append(current_age_group)
        
        # Extract sex
        sex = get_sex(current_patient_data)
        sex_features = np.zeros(2, dtype=int)
        if compare_strings(sex, 'Female'):
            sex_features[0] = 1
        elif compare_strings(sex, 'Male'):
            sex_features[1] = 1
        features['sex'].append(sex_features)
        
        # Extract height and weight.
        height = get_height(current_patient_data)
        weight = get_weight(current_patient_data)
        features['hw'].append(np.array([height, weight]))
        
        # Extract pregnancy
        is_pregnant = get_pregnancy_status(current_patient_data)
        features['preg'].append(is_pregnant)

        # Extract location
        locations = entries[0]
        num_recording_locations = len(recording_locations)
        loc_features = np.zeros(num_recording_locations)
        if locations in recording_locations:
            j = recording_locations.index(locations)
            loc_features[j] = 1
        features['loc'].append(loc_features)

        # Extract labels and use one-hot encoding.
        current_labels = np.zeros(num_classes, dtype=int)
        label = get_label(current_patient_data)
        if label in classes:
            j = classes.index(label)
            current_labels[j] = 1
        labels.append(current_labels)

In [145]:
len(labels)

2675

In [ ]:
labels = np.array(labels)

## Modeling

In [64]:
from tensorflow import keras
from tensorflow.keras import layers

In [65]:
len(age_classes)

5

In [66]:
mel1.shape

(100, 313)

In [98]:
features_trn[0]['mel1'][0].shape

(100, 313, 1)

In [69]:
age = keras.Input(shape=(6,), name = 'age_cat')
sex = keras.Input(shape=(2,), name = 'sex_cat')
hw = keras.Input(shape=(2,), name = 'height_weight')
preg = keras.Input(shape=(1,), name = 'is_preg')
loc = keras.Input(shape=(len(recording_locations),), name = 'loc')
mel1 = keras.Input(shape=((100, 313, 1)), name = 'mel')


In [70]:
features.keys()

dict_keys(['id', 'age', 'sex', 'hw', 'preg', 'loc', 'mel1'])

## A toy model

In [73]:
## age embeddig
age1 = layers.Dense(2, activation = 'relu')(age)

## sex embedding
sex1 = layers.Dense(1, activation = 'relu')(sex)

## hw embedding
hw1 = layers.Dense(1, activation = 'relu')(hw)

## loc embedding
loc1 = layers.Dense(3, activation = 'relu')(loc)

## mel embedding
mel2 = layers.Conv2D(16, (3,3), activation = 'relu')(mel1)
mel2 = layers.MaxPooling2D()(mel2)
mel2 = layers.Conv2D(32, (5,5), activation = 'relu')(mel2)
mel2 = layers.MaxPooling2D()(mel2)
mel2 = layers.Conv2D(32, (3,3), activation = 'relu')(mel2)
mel2 = layers.MaxPooling2D()(mel2)
mel2 = layers.Conv2D(64, (3,3), activation = 'relu')(mel2)
mel2 = layers.MaxPooling2D()(mel2)
mel2 = layers.GlobalAveragePooling2D()(mel2)

concat1 = layers.Concatenate()([age1, sex1, hw1, loc1, mel2, preg])
concat1 = layers.Dense(10, activation = 'relu')(concat1)
concat2 = layers.Dense(2, activation = "softmax")(concat1)
concat3 = layers.Dense(2, activation = "softmax")(concat1)



In [74]:
model = keras.Model(inputs = [age,sex,hw,preg,loc,mel1] , outputs = [concat2, concat3] )


In [75]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mel (InputLayer)                [(None, 100, 313, 1) 0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 98, 311, 16)  160         mel[0][0]                        
__________________________________________________________________________________________________
max_pooling2d_8 (MaxPooling2D)  (None, 49, 155, 16)  0           conv2d_8[0][0]                   
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 45, 151, 32)  12832       max_pooling2d_8[0][0]            
______________________________________________________________________________________________

In [183]:
def get_toy(mel_input_shape):
        # Create a towy model.
    age = keras.Input(shape=(6,), name = 'age_cat')
    sex = keras.Input(shape=(2,), name = 'sex_cat')
    hw = keras.Input(shape=(2,), name = 'height_weight')
    preg = keras.Input(shape=(1,), name = 'is_preg')
    loc = keras.Input(shape=(5,), name = 'loc')
    mel1 = keras.Input(shape=(mel_input_shape), name = 'mel')
        
    ## age embeddig
    age1 = layers.Dense(2, activation = 'relu')(age)

    ## sex embedding
    sex1 = layers.Dense(1, activation = 'relu')(sex)

    ## hw embedding
    hw1 = layers.Dense(1, activation = 'relu')(hw)

    ## loc embedding
    loc1 = layers.Dense(3, activation = 'relu')(loc)

    ## mel embedding
    mel2 = layers.Conv2D(16, (3,3), activation = 'relu')(mel1)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(32, (5,5), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(32, (3,3), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(64, (3,3), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.GlobalAveragePooling2D()(mel2)

    concat1 = layers.Concatenate()([age1, sex1, hw1, loc1, mel2, preg])
    concat1 = layers.Dense(10, activation = 'relu')(concat1)
#    concat1 = layers.Dense(3, activation = "softmax")(concat1)
    concat2 = layers.Dense(2, activation = "softmax")(concat1)
#    concat3 = layers.Dense(2, activation = "softmax")(concat1)

    model = keras.Model(inputs = [age,sex,hw,preg,loc,mel1] , outputs = concat2 )
    
    model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])
    return(model)

In [221]:
def get_toy2(mel_input_shape):
        # Create a towy model.
    age = keras.Input(shape=(6,), name = 'age_cat')
    sex = keras.Input(shape=(2,), name = 'sex_cat')
    hw = keras.Input(shape=(2,), name = 'height_weight')
    preg = keras.Input(shape=(1,), name = 'is_preg')
    loc = keras.Input(shape=(5,), name = 'loc')
    mel1 = keras.Input(shape=(mel_input_shape), name = 'mel')
        
    ## age embeddig
    age1 = layers.Dense(2, activation = 'relu')(age)

    ## sex embedding
    sex1 = layers.Dense(1, activation = 'relu')(sex)

    ## hw embedding
    hw1 = layers.Dense(1, activation = 'relu')(hw)

    ## loc embedding
    loc1 = layers.Dense(3, activation = 'relu')(loc)

    ## mel embedding
    mel2 = layers.Conv2D(16, (3,3), activation = 'relu')(mel1)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(32, (5,5), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(32, (3,3), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(64, (3,3), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.GlobalAveragePooling2D()(mel2)

    concat1 = layers.Concatenate()([age1, sex1, hw1, loc1, mel2, preg])
    concat1 = layers.Dense(10, activation = 'relu')(concat1)
#    concat1 = layers.Dense(3, activation = "softmax")(concat1)
    concat2 = layers.Dense(3, activation = "softmax")(concat1)
    concat3 = layers.Dense(2, activation = "softmax")(concat1)

    model = keras.Model(inputs = [age,sex,hw,preg,loc,mel1] , outputs = [concat2, concat3] )
    
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])
    return(model)

In [77]:
model.compile(optimizer = "adam", 
             loss = ['categorical_crossentropy', 'categorical_crossentropy'],
             metrics = "accuracy")

In [99]:
features_trn[0]['sex']

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]])

In [100]:
features_trn[1].shape

(751, 2)

In [101]:
features_trn[2].shape

(751, 2)

In [224]:
model = get_toy2((100, 313, 1))

In [225]:
features_trn[0]['hw']

array([[98. , 15.9],
       [98. , 15.9],
       [98. , 15.9],
       ...,
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan]])

In [170]:
from sklearn.impute import SimpleImputer

In [226]:
imputer = SimpleImputer().fit(features_trn[0]['hw'])

In [227]:
features_trn[0]['hw'] = imputer.transform(features_trn[0]['hw'])
features_test[0]['hw'] = imputer.transform(features_test[0]['hw'])

In [228]:
features_trn[0]['hw']

array([[ 98.        ,  15.9       ],
       [ 98.        ,  15.9       ],
       [ 98.        ,  15.9       ],
       ...,
       [115.48300537,  25.12023681],
       [115.48300537,  25.12023681],
       [115.48300537,  25.12023681]])

In [229]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mel (InputLayer)                [(None, 100, 313, 1) 0                                            
__________________________________________________________________________________________________
conv2d_52 (Conv2D)              (None, 98, 311, 16)  160         mel[0][0]                        
__________________________________________________________________________________________________
max_pooling2d_52 (MaxPooling2D) (None, 49, 155, 16)  0           conv2d_52[0][0]                  
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 45, 151, 32)  12832       max_pooling2d_52[0][0]           
___________________________________________________________________________________________

In [230]:
model.fit([features_trn[0]['age'],features_trn[0]['sex'], features_trn[0]['hw'], features_trn[0]['preg'], features_trn[0]['loc'], 
           features_trn[0]['mel1']], [features_trn[1],features_trn[2]],
          validation_data = ([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], 
                              features_test[0]['preg'], features_test[0]['loc'], features_test[0]['mel1']], 
                             [features_test[1], features_test[2]]), 
          epochs = 30)

Epoch 1/30
80/80 [==============================] - 18s 231ms/step - loss: 1.3228 - dense_91_loss: 0.6410 - dense_92_loss: 0.6818 - dense_91_accuracy: 0.7492 - dense_92_accuracy: 0.5249 - val_loss: 1.2967 - val_dense_91_loss: 0.6085 - val_dense_92_loss: 0.6882 - val_dense_91_accuracy: 0.7448 - val_dense_92_accuracy: 0.5024
Epoch 2/30
80/80 [==============================] - 18s 229ms/step - loss: 1.2681 - dense_91_loss: 0.5917 - dense_92_loss: 0.6764 - dense_91_accuracy: 0.7622 - dense_92_accuracy: 0.5324 - val_loss: 1.2790 - val_dense_91_loss: 0.6068 - val_dense_92_loss: 0.6722 - val_dense_91_accuracy: 0.7448 - val_dense_92_accuracy: 0.5135
Epoch 3/30
80/80 [==============================] - 18s 229ms/step - loss: 1.2655 - dense_91_loss: 0.5924 - dense_92_loss: 0.6731 - dense_91_accuracy: 0.7595 - dense_92_accuracy: 0.5201 - val_loss: 1.2740 - val_dense_91_loss: 0.6061 - val_dense_92_loss: 0.6679 - val_dense_91_accuracy: 0.7544 - val_dense_92_accuracy: 0.5008
Epoch 4/30
80/80 [=======

80/80 [==============================] - 18s 229ms/step - loss: 0.9407 - dense_91_loss: 0.3546 - dense_92_loss: 0.5860 - dense_91_accuracy: 0.8760 - dense_92_accuracy: 0.6619 - val_loss: 1.0393 - val_dense_91_loss: 0.4167 - val_dense_92_loss: 0.6226 - val_dense_91_accuracy: 0.8574 - val_dense_92_accuracy: 0.6355
Epoch 27/30
80/80 [==============================] - 18s 229ms/step - loss: 0.9456 - dense_91_loss: 0.3597 - dense_92_loss: 0.5859 - dense_91_accuracy: 0.8752 - dense_92_accuracy: 0.6647 - val_loss: 1.0203 - val_dense_91_loss: 0.4201 - val_dense_92_loss: 0.6001 - val_dense_91_accuracy: 0.8494 - val_dense_92_accuracy: 0.6418
Epoch 28/30
80/80 [==============================] - 18s 229ms/step - loss: 0.9449 - dense_91_loss: 0.3574 - dense_92_loss: 0.5876 - dense_91_accuracy: 0.8776 - dense_92_accuracy: 0.6564 - val_loss: 1.0292 - val_dense_91_loss: 0.4174 - val_dense_92_loss: 0.6118 - val_dense_91_accuracy: 0.8479 - val_dense_92_accuracy: 0.6434
Epoch 29/30
80/80 [===============

In [231]:
import pickle as pk
def save_challenge_model2(model_folder, model, m_name, mel_shape = (100, 313, 1)) :
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename = os.path.join(model_folder, m_name + '_model.hdf5')
    model.save(filename)
    d = {'model': m_name, 'mel_shape': mel_shape, 'model_fnm': filename}    
    with open(info_fnm, 'wb') as f:
        pk.dump(d, f, pk.HIGHEST_PROTOCOL)
    return 1
    

In [232]:
# Save the model.
save_challenge_model2(model_folder, model, m_name = 'toy')

1

In [233]:
def load_challenge_model(model_folder, verbose):
    info_fnm = os.path.join(model_folder, 'desc.pk')
    with open(info_fnm, 'rb') as f:
        info_m = pk.load(f)
#    if info_m['model'] == 'toy' :
#        model = get_toy(info_m['mel_shape'])
#    filename = os.path.join(model_folder, info_m['model'] + '_model.hdf5')
#    model.load_weights(filename)
    return info_m

def get_toy(mel_input_shape):
        # Create a towy model.
    age = keras.Input(shape=(6,), name = 'age_cat')
    sex = keras.Input(shape=(2,), name = 'sex_cat')
    hw = keras.Input(shape=(2,), name = 'height_weight')
    preg = keras.Input(shape=(1,), name = 'is_preg')
    loc = keras.Input(shape=(5,), name = 'loc')
    mel1 = keras.Input(shape=(mel_input_shape), name = 'mel')
        
    ## age embeddig
    age1 = layers.Dense(2, activation = 'relu')(age)

    ## sex embedding
    sex1 = layers.Dense(1, activation = 'relu')(sex)

    ## hw embedding
    hw1 = layers.Dense(1, activation = 'relu')(hw)

    ## loc embedding
    loc1 = layers.Dense(3, activation = 'relu')(loc)

    ## mel embedding
    mel2 = layers.Conv2D(16, (3,3), activation = 'relu')(mel1)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(32, (5,5), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(32, (3,3), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(64, (3,3), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.GlobalAveragePooling2D()(mel2)

    concat1 = layers.Concatenate()([age1, sex1, hw1, loc1, mel2, preg])
    concat1 = layers.Dense(10, activation = 'relu')(concat1)
    concat1 = layers.Dense(3, activation = "softmax")(concat1)
    model = keras.Model(inputs = [age,sex,hw,preg,loc,mel1] , outputs = concat1 )
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return(model)

In [120]:
save_challenge_model2(model_folder, model, m_name = 'toy')

1

In [234]:
model2 = load_challenge_model(model_folder, verbose = 1) ### Teams: Implement this function!!!


In [235]:
model2

{'model': 'toy',
 'mel_shape': (100, 313, 1),
 'model_fnm': 'tmp_model2/toy_model.hdf5'}

In [236]:
patient_data = load_patient_data(patient_files_test[0])
recordings = load_recordings(data_folder, patient_data)

In [129]:
data = patient_data

In [ ]:
def run_challenge_model(model, data, recordings, verbose):
    
    if model['model'] == 'toy' :
        model1 = get_toy(model['mel_shape'])
    model1.load_weights(model['model_fnm'])
    
    classes = model['classes']
    # Load features.
    features = get_feature_one(data, verbose = 0)

    features['mel1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_melspec(recordings[i])[0]
        features['mel1'].append(mel1)

    M, N = features['mel1'][0].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)   
        
    features['mel1'] = np.array(features['mel1'])
#    print(features)
    # Impute missing data.
    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1']])

    # Get classifier probabilities.
    prob1 = res1.mean(axis = 0) ## simple rule for now
    idx = np.argmax(prob1)
    # Choose label with higher probability.
    labels = np.zeros(len(classes), dtype=np.int_)
    labels[idx] = 1

    return classes, labels, prob1

In [237]:
model1

{'model': 'toy',
 'mel_shape': (100, 313, 1),
 'model_fnm': 'tmp_model2/toy_model.hdf5'}

In [238]:
    if model1['model'] == 'toy' :
        model2 = get_toy2(model1['mel_shape'])
    model2.load_weights(model1['model_fnm'])
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Normal', 'Abnormal']
    # Load features.
    features = get_feature_one(data, verbose = 0)
    features['mel1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_melspec(recordings[i])[0]
        features['mel1'].append(mel1)

    M, N = features['mel1'][0].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)   
        
    features['mel1'] = np.array(features['mel1'])
    res1 = model2.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1']])
    res1

In [241]:
    res1 = model2.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1']])


In [242]:
res1

[array([[2.2546257e-16, 1.2601602e-16, 1.0000000e+00],
        [6.9195350e-16, 3.9185369e-16, 1.0000000e+00],
        [1.3673014e-16, 7.5975606e-17, 1.0000000e+00],
        [4.0297731e-17, 2.2074177e-17, 1.0000000e+00]], dtype=float32),
 array([[1.7146952e-09, 1.0000000e+00],
        [3.1972824e-09, 1.0000000e+00],
        [1.2986671e-09, 1.0000000e+00],
        [6.5870259e-10, 1.0000000e+00]], dtype=float32)]

In [243]:
patient_data

'33151 4 4000\nAV 33151_AV.hea 33151_AV.wav 33151_AV.tsv\nPV 33151_PV.hea 33151_PV.wav 33151_PV.tsv\nTV 33151_TV.hea 33151_TV.wav 33151_TV.tsv\nMV 33151_MV.hea 33151_MV.wav 33151_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 141.0\n#Weight: 30.9\n#Pregnancy status: False\n#Murmur: Present\n#Murmur locations: MV+TV\n#Most audible location: TV\n#Systolic murmur timing: Holosystolic\n#Systolic murmur shape: Plateau\n#Systolic murmur grading: I/VI\n#Systolic murmur pitch: Low\n#Systolic murmur quality: Harsh\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Abnormal\n#Campaign: CC2015\n#Additional ID: nan\n'

In [ ]:
    if model1['model'] == 'toy' :
        model2 = get_toy2(model1['mel_shape'])
    model2.load_weights(model1['model_fnm'])
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Normal', 'Abnormal']
    # Load features.
    features = get_feature_one(data, verbose = 0)

    features['mel1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_melspec(recordings[i])[0]
        features['mel1'].append(mel1)

    M, N = features['mel1'][0].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)   
        
    features['mel1'] = np.array(features['mel1'])
#    print(features)
    # Impute missing data.
    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1']])

    # Get classifier probabilities.
    prob1 = res1.mean(axis = 0) ## simple rule for now
    idx = np.argmax(prob1)
    # Choose label with higher probability.
    labels = np.zeros(len(classes), dtype=np.int_)
    labels[idx] = 1


In [125]:
def get_feature_one(patient_data, verbose = 0) :

    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
    
    num_locations = get_num_locations(patient_data)
    recording_information = patient_data.split('\n')[1:num_locations+1]

    features = dict()
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
#    features['mel1'] = []
    for j in range(num_locations) :
        entries = recording_information[j].split(' ')
        recording_file = entries[2]
#        filename = os.path.join(data_folder, recording_file)

        # Extract id
    #    id1 = recording_file.split('_')[0]
    #    features['id'].append(id1)

        # Extract melspec
#        mel1 = feature_extract_melspec(filename)[0]
#        features['mel1'].append(mel1)

        # Extract age_group
        age_group = get_age(patient_data)
        current_age_group = np.zeros(6, dtype=int)
        if age_group in age_classes:
            j = age_classes.index(age_group)
            current_age_group[j] = 1
        else :
            current_age_group[5] = 1
        features['age'].append(current_age_group)

        # Extract sex
        sex = get_sex(patient_data)
        sex_features = np.zeros(2, dtype=int)
        if compare_strings(sex, 'Female'):
            sex_features[0] = 1
        elif compare_strings(sex, 'Male'):
            sex_features[1] = 1
        features['sex'].append(sex_features)

        # Extract height and weight.
        height = get_height(patient_data)
        weight = get_weight(patient_data)
        features['hw'].append(np.array([height, weight]))

        # Extract pregnancy
        is_pregnant = get_pregnancy_status(patient_data)
        features['preg'].append(is_pregnant)

        # Extract location
        locations = entries[0]
        num_recording_locations = len(recording_locations)
        loc_features = np.zeros(num_recording_locations)
        if locations in recording_locations:
            j = recording_locations.index(locations)
            loc_features[j] = 1
        features['loc'].append(loc_features)
        

    for k1 in features.keys() :
        features[k1] = np.array(features[k1])
        
    if verbose :
        label = get_label(patient_data)
        print(label)
    return features

In [128]:
patient_data

'33151 4 4000\nAV 33151_AV.hea 33151_AV.wav 33151_AV.tsv\nPV 33151_PV.hea 33151_PV.wav 33151_PV.tsv\nTV 33151_TV.hea 33151_TV.wav 33151_TV.tsv\nMV 33151_MV.hea 33151_MV.wav 33151_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 141.0\n#Weight: 30.9\n#Pregnancy status: False\n#Murmur: Present\n#Murmur locations: MV+TV\n#Most audible location: TV\n#Systolic murmur timing: Holosystolic\n#Systolic murmur shape: Plateau\n#Systolic murmur grading: I/VI\n#Systolic murmur pitch: Low\n#Systolic murmur quality: Harsh\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Abnormal\n#Campaign: CC2015\n#Additional ID: nan\n'

In [130]:
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Normal', 'Abnormal']
num_mm_classes = len(murmur_classes)
num_o_classes = len(outcome_classes)

if model1['model'] == 'toy' :
    model2 = get_toy(model1['mel_shape'])
model2.load_weights(model1['model_fnm'])

features = get_feature_one(data, verbose = 0)

In [131]:
recordings

[array([ -455,  1009,   729, ..., -2099, -1931, -1825], dtype=int16),
 array([1174, 4679, 5900, ...,  -36, -171,  -73], dtype=int16),
 array([7710, 5756, 2397, ...,  950,  784,  923], dtype=int16),
 array([14951,  7497,  -352, ...,  1297,   811,    61], dtype=int16)]

In [132]:
features['mel1'] = []
for i in range(len(recordings)) :
    mel1 = feature_extract_melspec(recordings[i])[0]
    features['mel1'].append(mel1)

In [133]:
M, N = features['mel1'][0].shape
for i in range(len(features['mel1'])) :
    features['mel1'][i] = features['mel1'][i].reshape(M,N,1) 

In [135]:
features['mel1'] = np.array(features['mel1'])
#    print(features)
# Impute missing data.
res1 = model2.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1']])

In [136]:
res1

[array([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], dtype=float32),
 array([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], dtype=float32)]

In [ ]:
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Normal', 'Abnormal']
num_mm_classes = len(murmur_classes)
num_o_classes = len(outcome_classes)

if model['model'] == 'toy' :
    model1 = get_toy(model['mel_shape'])
model1.load_weights(model['model_fnm'])

features = get_feature_one(data, verbose = 0)


#imputer = model['imputer']
classifier = model['classifier']

if model['model'] == 'toy' :
    model1 = get_toy(info_m['mel_shape'])
filename = os.path.join(model_folder, info_m['model'] + '_model.hdf5')
model.load_weights(filename)

# Load features.
features = get_features(data, recordings)

# Impute missing data.
features = features.reshape(1, -1)
features = imputer.transform(features)

# Get classifier probabilities.
probabilities = classifier.predict_proba(features)
probabilities = np.asarray(probabilities, dtype=np.float32)[:, 0, 1]

# Choose label with higher probability.
labels = np.zeros(len(classes), dtype=np.int_)
idx = np.argmax(probabilities)
labels[idx] = 1


In [393]:
classes, labels, probabilities = run_challenge_model(model1, patient_data, recordings, verbose) ### Teams: Implement this function!!!


NameError: name 'run_challenge_model' is not defined

In [333]:
num_locations = get_num_locations(patient_data)
recording_information = current_patient_data.split('\n')[1:num_locations+1]


In [336]:
features = dict()
features['age'] = []
features['sex'] = []
features['hw'] = []
features['preg'] = []
features['loc'] = []
features['mel1'] = []
for j in range(num_locations) :
    entries = recording_information[j].split(' ')
    recording_file = entries[2]
    filename = os.path.join(data_folder, recording_file)

    # Extract id
#    id1 = recording_file.split('_')[0]
#    features['id'].append(id1)

    # Extract melspec
    mel1 = feature_extract_melspec(filename)[0]
    features['mel1'].append(mel1)

    # Extract age_group
    age_group = get_age(current_patient_data)
    current_age_group = np.zeros(6, dtype=int)
    if age_group in age_classes:
        j = age_classes.index(age_group)
        current_age_group[j] = 1
    else :
        current_age_group[5] = 1
    features['age'].append(current_age_group)

    # Extract sex
    sex = get_sex(current_patient_data)
    sex_features = np.zeros(2, dtype=int)
    if compare_strings(sex, 'Female'):
        sex_features[0] = 1
    elif compare_strings(sex, 'Male'):
        sex_features[1] = 1
    features['sex'].append(sex_features)

    # Extract height and weight.
    height = get_height(current_patient_data)
    weight = get_weight(current_patient_data)
    features['hw'].append(np.array([height, weight]))

    # Extract pregnancy
    is_pregnant = get_pregnancy_status(current_patient_data)
    features['preg'].append(is_pregnant)

    # Extract location
    locations = entries[0]
    num_recording_locations = len(recording_locations)
    loc_features = np.zeros(num_recording_locations)
    if locations in recording_locations:
        j = recording_locations.index(locations)
        loc_features[j] = 1
    features['loc'].append(loc_features)

In [342]:
    M, N = features['mel1'][0].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)

    for k1 in features.keys() :
        features[k1] = np.array(features[k1])

In [346]:
features.keys()

dict_keys(['age', 'sex', 'hw', 'preg', 'loc', 'mel1'])

In [348]:
res1 = model.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1']])

In [352]:
res1.argmax(axis=1)

array([2, 2, 2, 2])

In [353]:
res1

array([[1.2032033e-03, 2.9272465e-03, 9.9586958e-01],
       [3.2206599e-02, 7.0056088e-02, 8.9773726e-01],
       [1.4683392e-04, 7.0860866e-04, 9.9914455e-01],
       [1.8912905e-03, 6.9301804e-03, 9.9117857e-01]], dtype=float32)

In [355]:
features_test[1][:4]

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [354]:
features_test[1][0]

array([1, 0, 0])

In [354]:
features_test[1][0]

array([1, 0, 0])

In [354]:
features_test[1][0]

array([1, 0, 0])

In [354]:
features_test[1][0]

array([1, 0, 0])

In [369]:
classes

['Present', 'Unknown', 'Absent']

In [81]:
def run_challenge_model(model, data, recordings, verbose):
    classes = model['classes']
    imputer = model['imputer']
    classifier = model['classifier']

    if model['model'] == 'toy' :
        model1 = get_toy(info_m['mel_shape'])
    filename = os.path.join(model_folder, info_m['model'] + '_model.hdf5')
    model.load_weights(filename)
    
    # Load features.
    features = get_features(data, recordings)

    # Impute missing data.
    features = features.reshape(1, -1)
    features = imputer.transform(features)

    # Get classifier probabilities.
    probabilities = classifier.predict_proba(features)
    probabilities = np.asarray(probabilities, dtype=np.float32)[:, 0, 1]

    # Choose label with higher probability.
    labels = np.zeros(len(classes), dtype=np.int_)
    idx = np.argmax(probabilities)
    labels[idx] = 1

    return classes, labels, probabilities

In [372]:
model

{'model': 'toy',
 'classes': ['Present', 'Unknown', 'Absent'],
 'mel_shape': (100, 313, 1)}

In [86]:
patient_data = load_patient_data(patient_files_test[1])
recordings = load_recordings(data_folder, patient_data)

In [87]:
features_test[1][:10]

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [88]:
patient_data

'85112 4 4000\nAV 85112_AV.hea 85112_AV.wav 85112_AV.tsv\nPV 85112_PV.hea 85112_PV.wav 85112_PV.tsv\nTV 85112_TV.hea 85112_TV.wav 85112_TV.tsv\nMV 85112_MV.hea 85112_MV.wav 85112_MV.tsv\n#Age: Child\n#Sex: Male\n#Height: 136.0\n#Weight: 26.3\n#Pregnancy status: False\n#Murmur: Absent\n#Murmur locations: nan\n#Most audible location: nan\n#Systolic murmur timing: nan\n#Systolic murmur shape: nan\n#Systolic murmur grading: nan\n#Systolic murmur pitch: nan\n#Systolic murmur quality: nan\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Campaign: CC2015\n#Additional ID: nan'

In [ ]:
classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!


In [89]:
classes

['Present', 'Unknown', 'Absent']

In [90]:
current_patient_data

'2530 4 4000\nAV 2530_AV.hea 2530_AV.wav 2530_AV.tsv\nPV 2530_PV.hea 2530_PV.wav 2530_PV.tsv\nTV 2530_TV.hea 2530_TV.wav 2530_TV.tsv\nMV 2530_MV.hea 2530_MV.wav 2530_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 98.0\n#Weight: 15.9\n#Pregnancy status: False\n#Murmur: Absent\n#Murmur locations: nan\n#Most audible location: nan\n#Systolic murmur timing: nan\n#Systolic murmur shape: nan\n#Systolic murmur grading: nan\n#Systolic murmur pitch: nan\n#Systolic murmur quality: nan\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Campaign: CC2015\n#Additional ID: nan'

In [91]:
num_locations = get_num_locations(patient_data)
recording_information = patient_data.split('\n')[1:num_locations+1]


In [92]:

features = dict()
features['age'] = []
features['sex'] = []
features['hw'] = []
features['preg'] = []
features['loc'] = []
features['mel1'] = []
for j in range(num_locations) :
    entries = recording_information[j].split(' ')
    recording_file = entries[2]
    filename = os.path.join(data_folder, recording_file)

    # Extract id
#    id1 = recording_file.split('_')[0]
#    features['id'].append(id1)

    # Extract melspec
    mel1 = feature_extract_melspec(filename)[0]
    features['mel1'].append(mel1)

    # Extract age_group
    age_group = get_age(patient_data)
    current_age_group = np.zeros(6, dtype=int)
    if age_group in age_classes:
        j = age_classes.index(age_group)
        current_age_group[j] = 1
    else :
        current_age_group[5] = 1
    features['age'].append(current_age_group)

    # Extract sex
    sex = get_sex(patient_data)
    sex_features = np.zeros(2, dtype=int)
    if compare_strings(sex, 'Female'):
        sex_features[0] = 1
    elif compare_strings(sex, 'Male'):
        sex_features[1] = 1
    features['sex'].append(sex_features)

    # Extract height and weight.
    height = get_height(patient_data)
    weight = get_weight(patient_data)
    features['hw'].append(np.array([height, weight]))

    # Extract pregnancy
    is_pregnant = get_pregnancy_status(patient_data)
    features['preg'].append(is_pregnant)

    # Extract location
    locations = entries[0]
    num_recording_locations = len(recording_locations)
    loc_features = np.zeros(num_recording_locations)
    if locations in recording_locations:
        j = recording_locations.index(locations)
        loc_features[j] = 1
    features['loc'].append(loc_features)

In [93]:
features

{'age': [array([0, 0, 1, 0, 0, 0]),
  array([0, 0, 1, 0, 0, 0]),
  array([0, 0, 1, 0, 0, 0]),
  array([0, 0, 1, 0, 0, 0])],
 'sex': [array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1])],
 'hw': [array([136. ,  26.3]),
  array([136. ,  26.3]),
  array([136. ,  26.3]),
  array([136. ,  26.3])],
 'preg': [False, False, False, False],
 'loc': [array([1., 0., 0., 0., 0.]),
  array([0., 0., 1., 0., 0.]),
  array([0., 0., 0., 1., 0.]),
  array([0., 1., 0., 0., 0.])],
 'mel1': [array([[  5.5544219 ,   5.25430448,   5.85624965, ...,  -4.01180172,
           -2.62854908,  -2.5042349 ],
         [ 10.42099442,   9.87717865,  14.38695231, ...,  -2.46004349,
            1.79924639,  -4.51349842],
         [ 11.33310424,  13.78394537,  13.1335616 , ...,   1.08339154,
           -1.01365032,  -7.04751297],
         ...,
         [-18.90015848, -54.90594607, -58.1136186 , ..., -59.42052707,
          -56.90998361, -48.85066669],
         [-15.42566047, -52.71301415, -60.73131942, ..., -58.35870

In [94]:
    M, N = features['mel1'][0].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)

    for k1 in features.keys() :
        features[k1] = np.array(features[k1])

In [85]:
def get_feature_one(patient_data, verbose = 0) :
    num_locations = get_num_locations(patient_data)
    recording_information = patient_data.split('\n')[1:num_locations+1]

    features = dict()
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
    features['mel1'] = []
    for j in range(num_locations) :
        entries = recording_information[j].split(' ')
        recording_file = entries[2]
        filename = os.path.join(data_folder, recording_file)

        # Extract id
    #    id1 = recording_file.split('_')[0]
    #    features['id'].append(id1)

        # Extract melspec
        mel1 = feature_extract_melspec(filename)[0]
        features['mel1'].append(mel1)

        # Extract age_group
        age_group = get_age(patient_data)
        current_age_group = np.zeros(6, dtype=int)
        if age_group in age_classes:
            j = age_classes.index(age_group)
            current_age_group[j] = 1
        else :
            current_age_group[5] = 1
        features['age'].append(current_age_group)

        # Extract sex
        sex = get_sex(patient_data)
        sex_features = np.zeros(2, dtype=int)
        if compare_strings(sex, 'Female'):
            sex_features[0] = 1
        elif compare_strings(sex, 'Male'):
            sex_features[1] = 1
        features['sex'].append(sex_features)

        # Extract height and weight.
        height = get_height(patient_data)
        weight = get_weight(patient_data)
        features['hw'].append(np.array([height, weight]))

        # Extract pregnancy
        is_pregnant = get_pregnancy_status(patient_data)
        features['preg'].append(is_pregnant)

        # Extract location
        locations = entries[0]
        num_recording_locations = len(recording_locations)
        loc_features = np.zeros(num_recording_locations)
        if locations in recording_locations:
            j = recording_locations.index(locations)
            loc_features[j] = 1
        features['loc'].append(loc_features)
        
        
    M, N = features['mel1'][0].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)

    for k1 in features.keys() :
        features[k1] = np.array(features[k1])
        
    if verbose :
        label = get_label(patient_data)
        print(label)
    return features

In [433]:
features.keys()

dict_keys(['age', 'sex', 'hw', 'preg', 'loc', 'mel1'])

In [95]:
res1 = model.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1']])

In [96]:
res1.argmax(axis=1)

array([0, 2, 2, 2])

In [100]:
prob1 = res1.mean(axis = 0)

In [101]:
probargmax = prob1.argmax()
labels = np.zeros((3,))
labels[probargmax] = 1

In [102]:
labels

array([0., 0., 1.])

In [103]:
classes

['Present', 'Unknown', 'Absent']

In [440]:
model1

{'model': 'toy',
 'classes': ['Present', 'Unknown', 'Absent'],
 'mel_shape': (100, 313, 1),
 'model_fnm': 'tmp_model1/toy_model.hdf5'}

In [84]:
model1

{'model': 'toy',
 'classes': ['Present', 'Unknown', 'Absent'],
 'mel_shape': (100, 313, 1),
 'model_fnm': 'tmp_model1/toy_model.hdf5'}

In [109]:
#classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
def run_challenge_model(model, data, recordings, verbose):
    
    
    
    if model['model'] == 'toy' :
        model1 = get_toy(model['mel_shape'])
    filename = os.path.join(model_folder, model['model'] + '_model.hdf5')
    model1.load_weights(filename)
    
    classes = model['classes']
    # Load features.
    features = get_feature_one(data, verbose)

    # Impute missing data.
    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1']])
    

    # Get classifier probabilities.
    prob1 = res1.mean(axis = 0) ## simple rule for now
    idx = np.argmax(prob1)
    # Choose label with higher probability.
    labels = np.zeros(len(classes), dtype=np.int_)
    labels[idx] = 1

    return classes, labels, prob1

In [105]:
len(patient_files_test)

142

In [110]:
run_challenge_model(model1, data = load_patient_data(patient_files_test[0]), recordings = 0, verbose = 1)

Present


(['Present', 'Unknown', 'Absent'],
 array([0, 0, 1]),
 array([0.42289853, 0.02808568, 0.5490158 ], dtype=float32))

In [111]:
run_challenge_model(model1, data = load_patient_data(patient_files_test[1]), recordings = 0, verbose = 1)

Absent


(['Present', 'Unknown', 'Absent'],
 array([0, 0, 1]),
 array([0.21346666, 0.03261768, 0.7539156 ], dtype=float32))

In [112]:
run_challenge_model(model1, data = load_patient_data(patient_files_test[2]), recordings = 0, verbose = 1)

Absent


(['Present', 'Unknown', 'Absent'],
 array([0, 0, 1]),
 array([0.11145334, 0.01811697, 0.8704297 ], dtype=float32))

In [113]:
run_challenge_model(model1, data = load_patient_data(patient_files_test[3]), recordings = 0, verbose = 1)

Absent


(['Present', 'Unknown', 'Absent'],
 array([0, 0, 1]),
 array([0.09037623, 0.00931931, 0.90030444], dtype=float32))

In [114]:
run_challenge_model(model1, data = load_patient_data(patient_files_test[4]), recordings = 0, verbose = 1)

Absent


(['Present', 'Unknown', 'Absent'],
 array([0, 0, 1]),
 array([0.04683066, 0.00709078, 0.94607854], dtype=float32))

In [115]:

ones1 = []
for i in range(100) :
    patient_data = load_patient_data(patient_files_test[i])
    num_locations = get_num_locations(patient_data)
    if(num_locations == 1) :
        ones1.append(i)
    

In [116]:
ones1

[24, 32, 80, 94, 98]

In [117]:
run_challenge_model(model1, data = load_patient_data(patient_files_test[24]), recordings = 0, verbose = 1)

Present


(['Present', 'Unknown', 'Absent'],
 array([0, 0, 1]),
 array([0.02266924, 0.06886302, 0.90846777], dtype=float32))

In [118]:
run_challenge_model(model1, data = load_patient_data(patient_files_test[32]), recordings = 0, verbose = 1)

Unknown


(['Present', 'Unknown', 'Absent'],
 array([0, 0, 1]),
 array([5.9274078e-04, 1.3342367e-02, 9.8606491e-01], dtype=float32))

In [119]:
run_challenge_model(model1, data = load_patient_data(patient_files_test[80]), recordings = 0, verbose = 1)

Absent


(['Present', 'Unknown', 'Absent'],
 array([0, 0, 1]),
 array([0.01424488, 0.0106928 , 0.97506225], dtype=float32))